In [133]:
import pandas as pd
import numpy as np
#import sweetviz as sv
import matplotlib.pyplot as plt
#import ipywidgets as widgets
#from datetime import datetime, timedelta, time
from bokeh import events
from bokeh.io import output_file, output_notebook, curdoc
from bokeh.plotting import figure, show
from bokeh.models import CustomJS, ColumnDataSource, LinearAxis, Range1d, Toggle, Slider, DatetimeTickFormatter, Select, Tabs, TabPanel, HoverTool, CrosshairTool, Span, Div, RadioGroup
from bokeh.layouts import row, column, gridplot, layout
#from bokeh.driving import count
output_file('layout1_v003.html')  # Render to static HTML, or
output_notebook()  # Render inline in a Jupyter Notebook
#from icecream import ic
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns',None)

Loading BokehJS ...

In [2]:
archivo = "../data/edificio_este/edificio_este.csv"

nombres= np.genfromtxt(archivo,max_rows=1,dtype="U",delimiter=',')

#for i,nombre in enumerate(nombres):
#    print(i,nombre)
ti_zones=[]
rh_zones=[]
all_zones=['N1AU401','N1AU402','N1AU403','N1AU404','N2AU101','N2AU102','N2AU103','N2AU201','N2AU202','N2AU203','N2PASILLO','N2STR','PBADM','PBATENCIONCOFI','PBCOORCOFI','PBCOORDINACION','PBDUCTOCOFI','PBDUCTOSALAS','PBDUCTOSCOM','PBIMP','PBSCOM','PBSJUNTAS','PBSMAESTROS','PBSTR','PBVESTIBULO']
zones1=['N1AU401','N1AU402','N1AU403','N1AU404','N2AU101','N2AU102','N2AU103','N2AU201','N2AU202','N2AU203','PBADM','PBATENCIONCOFI','PBCOORCOFI','PBCOORDINACION','PBSJUNTAS','PBSMAESTROS','PBVESTIBULO']
zMAT=range(2,27,1)
zRAH=range(27,52,1)
nombres[0]='time'
nombres[1]='T_out'
for i in range(len(all_zones)):
    nombres[zMAT[i]]='Ti_'+all_zones[i]
    nombres[zRAH[i]]='RH_'+all_zones[i]
    if all_zones[i] in zones1:
        ti_zones.append(nombres[zMAT[i]])
        rh_zones.append(nombres[zRAH[i]])
        
use_cols=[nombres[0],nombres[1]]+ti_zones+rh_zones
def eplus_fileimport(archivo):
    tmp = pd.read_csv(archivo,names=nombres,skiprows=1,usecols=use_cols)
    tmp.time = tmp.time.str.replace('24:00:00','23:59:59')
    tmp.time = '2019 '+ tmp.time
    tmp.time = pd.to_datetime(tmp.time,format='%Y %m/%d %H:%M:%S')
    tmp.set_index('time',inplace=True)
    return tmp

simulation = eplus_fileimport(archivo)

este = simulation.truncate(after='2019-01-01 23:59:59')

archivo2 = "../data/edificio_oeste/edificio_oeste.csv"

nombres2= np.genfromtxt(archivo,max_rows=1,dtype="U",delimiter=',')


ti_zones=[]
rh_zones=[]
# all_zones=['N1AU401','N1AU402','N1AU403','N1AU404','N2AU101','N2AU102','N2AU103','N2AU201','N2AU202','N2AU203','N2PASILLO','N2STR','PBADM','PBATENCIONCOFI','PBCOORCOFI','PBCOORDINACION','PBDUCTOCOFI','PBDUCTOSALAS','PBDUCTOSCOM','PBIMP','PBSCOM','PBSJUNTAS','PBSMAESTROS','PBSTR','PBVESTIBULO']
zones2=['N1LAB','N2LAB1','N2LAB2','PBCAFE','PBCOCINA']
zMAT=range(2,7,1)
zRAH=range(7,12,1)
nombres2[0]='time'
for i in range(len(zones2)):
    nombres2[zMAT[i]]='Ti_'+zones2[i]
    nombres2[zRAH[i]]='RH_'+zones2[i]
    ti_zones.append(nombres2[zMAT[i]])
    rh_zones.append(nombres2[zRAH[i]])
# for i,nombre in enumerate(nombres2):
#    print(i,nombre)

use_cols=[nombres2[0]]+ti_zones+rh_zones
def eplus_fileimport(archivo):
    tmp = pd.read_csv(archivo,names=nombres2,skiprows=1,usecols=use_cols)
    tmp.time = tmp.time.str.replace('24:00:00','23:59:59')
    tmp.time = '2019 '+ tmp.time
    tmp.time = pd.to_datetime(tmp.time,format='%Y %m/%d %H:%M:%S')
    tmp['datetime']=tmp.time
    tmp.set_index('time',inplace=True)
    tmp['hour']=tmp.index.hour
    return tmp

simulation = eplus_fileimport(archivo)

oeste = simulation.truncate(after='2019-01-01 23:59:59')

zones=zones1+zones2

edificio = pd.merge(este,oeste,left_index=True,right_index=True)
edificio

,T_out,Ti_N1AU401,Ti_N1AU402,Ti_N1AU403,Ti_N1AU404,Ti_N2AU101,Ti_N2AU102,Ti_N2AU103,Ti_N2AU201,Ti_N2AU202,Ti_N2AU203,Ti_PBADM,Ti_PBATENCIONCOFI,Ti_PBCOORCOFI,Ti_PBCOORDINACION,Ti_PBSJUNTAS,Ti_PBSMAESTROS,Ti_PBVESTIBULO,RH_N1AU401,RH_N1AU402,RH_N1AU403,RH_N1AU404,RH_N2AU101,RH_N2AU102,RH_N2AU103,RH_N2AU201,RH_N2AU202,RH_N2AU203,RH_PBADM,RH_PBATENCIONCOFI,RH_PBCOORCOFI,RH_PBCOORDINACION,RH_PBSJUNTAS,RH_PBSMAESTROS,RH_PBVESTIBULO,Ti_N1LAB,Ti_N2LAB1,Ti_N2LAB2,Ti_PBCAFE,Ti_PBCOCINA,RH_N1LAB,RH_N2LAB1,RH_N2LAB2,RH_PBCAFE,RH_PBCOCINA,datetime,hour
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:10:00,19.0,20.726782,20.743574,20.716925,20.714329,21.155496,21.210239,21.177017,20.969827,21.013884,20.995471,22.725073,22.331225,22.978789,22.218651,22.178354,22.772624,22.784297,33.311427,33.314091,33.346907,33.357034,32.276501,32.209122,32.248266,32.786049,32.732366,32.745479,27.943910,29.758499,27.065981,30.195349,30.076049,27.556078,28.188637,20.726782,20.743574,20.716925,20.714329,21.155496,21.210239,21.177017,20.969827,21.013884,20.995471,2019-01-01 00:10:00,0
2019-01-01 00:20:00,18.9,20.527649,20.565701,20.525493,20.525053,21.039569,21.099529,21.062909,20.913115,20.953221,20.938859,22.702370,22.473988,22.880506,22.216611,22.175611,22.718851,22.768629,32.626961,32.621835,32.658325,32.667221,31.603967,31.557549,31.582919,32.201527,32.187873,32.171211,27.962516,29.406591,27.239201,30.061051,29.966217,27.662581,28.209501,20.527649,20.565701,20.525493,20.525053,21.039569,21.099529,21.062909,20.913115,20.953221,20.938859,2019-01-01 00:20:00,0
2019-01-01 00:30:00,18.8,20.071823,20.124932,20.079401,20.079515,20.975666,21.035143,20.998733,21.211742,21.224636,21.225054,22.632499,22.499120,22.791874,22.005029,21.998984,22.731341,22.670909,33.491043,33.426171,33.491618,33.496191,31.776185,31.711855,31.748884,31.706393,31.732652,31.694838,28.093225,29.367607,27.407531,30.394919,30.255234,27.649232,28.369958,20.071823,20.124932,20.079401,20.079515,20.975666,21.035143,20.998733,21.211742,21.224636,21.225054,2019-01-01 00:30:00,0
2019-01-01 00:40:00,18.8,19.836974,19.882054,19.844287,19.842548,20.968006,21.023613,20.989670,21.189177,21.218702,21.208373,22.558348,22.508038,22.733678,21.879450,21.884639,22.733031,22.601462,33.955206,33.880843,33.946672,33.952320,31.826973,31.751190,31.795705,31.764484,31.749219,31.737914,28.238913,29.369267,27.535952,30.553990,30.420528,27.658592,28.487217,19.836974,19.882054,19.844287,19.842548,20.968006,21.023613,20.989670,21.189177,21.218702,21.208373,2019-01-01 00:40:00,0
2019-01-01 00:50:00,19.2,19.973575,19.996664,19.973581,19.968415,21.172392,21.215561,21.189698,21.099552,21.144372,21.126363,22.533279,22.562738,22.683747,21.944993,21.928253,22.718371,22.581623,33.535621,33.500014,33.539603,33.551445,31.313612,31.255111,31.288512,31.784792,31.738302,31.743011,28.295597,29.270107,27.654557,30.351392,30.283710,27.696862,28.511418,19.973575,19.996664,19.973581,19.968415,21.172392,21.215561,21.189698,21.099552,21.144372,21.126363,2019-01-01 00:50:00,0
2019-01-01 01:00:00,19.2,20.159955,20.175097,20.157359,20.150168,21.279107,21.324003,21.297057,21.082037,21.121406,21.108408,22.547816,22.618989,22.649075,22.066769,22.030592,22.692289,22.589149,33.414572,33.372416,33.415867,33.429633,31.210987,31.131678,31.178585,31.777225,31.732038,31.734001,28.290925,29.157245,27.769901,30.093453,30.076258,27.756749,28.504853,20.159955,20.175097,20.157359,20.150168,21.279107,21.324003,21.297057,21.082037,21.121406,21.108408,2019-01-01 01:00:00,1
2019-01-01 01:10:00,19.2,20.380986,20.405270,20.380831,20.375231,21.192932,21.242498,21.212438,21.087764,21.123757,21.112151,22.565229,22.604109,22.649775,22.074318,22.044057,22.679004,22.584066,33.125388,33.059826,33.119912,33.129806,31.504924,31.409045,31.466871,31.836774,31.790955,31.795347,28.267569,29.180480,27.792401,30.095153,30.066243,27.786994,28.526252,20.380986,20.405270,20.380831,20.375231,21.192932,21.242498,21.212438,21.087764,21.123757,21.112151,2019-01-01 01:

In [3]:
zones

['N1AU401',
 'N1AU402',
 'N1AU403',
 'N1AU404',
 'N2AU101',
 'N2AU102',
 'N2AU103',
 'N2AU201',
 'N2AU202',
 'N2AU203',
 'PBADM',
 'PBATENCIONCOFI',
 'PBCOORCOFI',
 'PBCOORDINACION',
 'PBSJUNTAS',
 'PBSMAESTROS',
 'PBVESTIBULO',
 'N1LAB',
 'N2LAB1',
 'N2LAB2',
 'PBCAFE',
 'PBCOCINA']

### Esquema de ventilación
Para llevar a cabo el análisis de ventilación tomaremos en cuenta que las ventanas son de 3x3m y hay 2 por Aula de cada lado, asimismo se considera que hay 1 de cada lado por oficina.

In [4]:
#Cálculo del área de ventanas
for zone in zones:
    if 'AU' in zone:
        effective_window_size=18
        tmp1=[]
        tmp2=[]
        for j in range(len(edificio)):
            if edificio.hour[j] > 18:
                tmp1.append(0)
                tmp2.append(0)
            elif edificio.hour[j] > 16:
                tmp1.append((0.5+np.random.uniform(-0.2,0.1))*effective_window_size)
                tmp2.append((0.5+np.random.uniform(-0.2,0.1))*effective_window_size)
            elif edificio.hour[j] > 13:
                tmp1.append((0.75+np.random.uniform(-0.1,0.2))*effective_window_size)
                tmp2.append((0.75+np.random.uniform(-0.1,0.2))*effective_window_size)
            elif edificio.hour[j] > 11:
                tmp1.append((0.5+np.random.uniform(-0.2,0.1))*effective_window_size)
                tmp2.append((0.5+np.random.uniform(-0.2,0.1))*effective_window_size)
            else:
                tmp1.append(0)
                tmp2.append(0)
        edificio['A_inW'+zone]=tmp1
        edificio['A_outW'+zone]=tmp2
        edificio['A_totW'+zone]=edificio['A_inW'+zone]+edificio['A_outW'+zone]
        #ic(tmp1)
        #ic(tmp2)
    else:
        effective_window_size=9
        tmp1=[]
        for j in range(len(edificio)):
            if edificio.hour[j] > 18:
                tmp1.append(0)
            elif edificio.hour[j] > 16:
                tmp1.append((0.5+np.random.uniform(-0.2,0.1))*effective_window_size)
            elif edificio.hour[j] > 13:
                tmp1.append((0.75+np.random.uniform(-0.1,0.2))*effective_window_size)
            elif edificio.hour[j] > 11:
                tmp1.append((0.5+np.random.uniform(-0.2,0.1))*effective_window_size)
            else:
                tmp1.append(0)
        edificio['A_totW'+zone]=tmp1

En este caso para el esquema de ventilación también consideramos que cada uno de los espacios tiene ventilas.
Estas tienen una área efectiva de 1.5x0.2m por espacio.
Primero calculamos las columnas de datos del ángulo de apertura, donde consideramos ángulos de 0° a 90° ó $\pi/2$

Debido a que buscamos que funcionen como sistema de ventilación nocturna, los consideramos prácticamente abiertos de 19:00h a 07:00h y cerrados en cualquier otro caso.

In [5]:
for zone in zones:
    tmp1=[]
    for j in range(len(edificio)):
        if edificio.hour[j] > 19:
            tmp1.append(np.random.uniform(0.97,1))
        elif edificio.hour[j] > 7:
            tmp1.append(0)
        else:
            tmp1.append(np.random.uniform(0.97,1))
    edificio['Ang_V'+zone]=tmp1

In [6]:
#Cálculo del área de ventilas
effective_vent_size=1.5*0.2
for zone in zones:
    edificio['A_totV'+zone]=edificio['Ang_V'+zone]*effective_vent_size

In [7]:
#Cálculo de la tasa de ventilación
#Relación de ventilación - Rv=A_in/A_out
#Cálculo de Efectividad aberturas
# for zone in zones:
#     cfr=[]
#     for j in range(len(edificio)):
#         if edificio['A_out'+zone][j]!=0 and edificio['A_in'+zone][j]!=0:
#             Rv=edificio['A_in'+zone][j]/edificio['A_out'+zone][j]
#             Cfr=(0.6*((Rv/np.sqrt(1+Rv**2))/np.sin(np.pi/4)))
#             cfr.append(Cfr)
#         else:
#             cfr.append(0)
#     edificio['Cfr_'+zone]=cfr
#https://core.ac.uk/download/pdf/48392421.pdf

In [8]:
edificio.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 144 entries, 2019-01-01 00:10:00 to 2019-01-01 23:59:59
Data columns (total 133 columns):
 #    Column                Dtype         
---   ------                -----         
 0    T_out                 float64       
 1    Ti_N1AU401            float64       
 2    Ti_N1AU402            float64       
 3    Ti_N1AU403            float64       
 4    Ti_N1AU404            float64       
 5    Ti_N2AU101            float64       
 6    Ti_N2AU102            float64       
 7    Ti_N2AU103            float64       
 8    Ti_N2AU201            float64       
 9    Ti_N2AU202            float64       
 10   Ti_N2AU203            float64       
 11   Ti_PBADM              float64       
 12   Ti_PBATENCIONCOFI     float64       
 13   Ti_PBCOORCOFI         float64       
 14   Ti_PBCOORDINACION     float64       
 15   Ti_PBSJUNTAS          float64       
 16   Ti_PBSMAESTROS        float64       
 17   Ti_PBVESTIBULO        float64     

In [9]:
edificio['Tn']=11.9+0.54*edificio.T_out.mean()
edificio['lsc']=edificio['Tn']+1.5
edificio['lic']=edificio['Tn']-1.5
GHDC = (edificio.Ti_PBVESTIBULO[edificio.Ti_PBVESTIBULO>edificio.lsc]-edificio.Tn).sum()*(1./6.)
GHDC

8.745675499993215

In [10]:
edificio['Tn']=13.5+0.54*edificio.T_out.mean()
edificio['lsc']=edificio['Tn']+1.5
edificio['lic']=edificio['Tn']-1.5
GHDC = (edificio.Ti_PBVESTIBULO[edificio.Ti_PBVESTIBULO>edificio.lsc]-edificio.Tn).sum()*(1./6.)
GHDC

0.0

In [11]:
edificio['Tn']=12.1+0.54*edificio.T_out.mean()
edificio['lsc']=edificio['Tn']+1.5
edificio['lic']=edificio['Tn']-1.5
GHDC = (edificio.Ti_PBVESTIBULO[edificio.Ti_PBVESTIBULO>edificio.lsc]-edificio.Tn).sum()*(1./6.)
GHDC

6.751218487304695

In [12]:
start_time=edificio.index[0].asm8
end_time=edificio.index[-1].asm8

In [148]:
#Data source 
source=ColumnDataSource(edificio)

#Figura 1 Temperatura al interior
#tooltips=[('Space','$name'),('Temperature','$y °C'),('Date','@x')]
height = Span(dimension="height", line_width=1)
# width = Span(dimension="width", line_width=0.5)
crosshair = CrosshairTool(overlay=height)
p = figure(width=1225, height=614,x_axis_type=None,margin=(0,600,0,0))
p.y_range = Range1d(16.,27.)
p.yaxis.axis_label = "Temperature [°C]"
hover1 = HoverTool(
    tooltips=[
        ('Space','$name'),
        #('Temperature','$y °C'),
        ('Hour','@datetime{%H:%M}')
    ]
)
#hover1.tooltips = [('Space','$name'),('Temperature','$y °C'),('Date','@$x')]
hover1.formatters = {'@datetime':'datetime'}
hover1.line_policy = 'nearest'
hover1.mode = 'mouse'
p.add_tools(hover1)
p.add_tools(crosshair)
p.line(y='Tn',x='time',source=source,line_color='orange',name='Tn',line_width=2,line_dash='dashdot')
p.line(y='lsc',x='time',source=source,line_color='red',line_dash='dashed')
p.line(y='lic',x='time',source=source,line_color='red',line_dash='dashed')
p.varea(x='time',y1=28,y2='lsc',source=source,fill_color='gray',fill_alpha=0.2)
p.varea(x='time',y1='lic',y2=10,source=source,fill_color='gray',fill_alpha=0.2)
for zona in zones:
    if 'AU' in zona:
        p.line(y='Ti_'+zona,x='time',source=source,line_color='blue',name=zona)
    elif 'LAB' in zona:
        p.line(y='Ti_'+zona,x='time',source=source,line_color='red',name=zona)
    else:
        p.line(y='Ti_'+zona,x='time',source=source,line_color='green',name=zona)
p.x_range.start=start_time
p.x_range.end=end_time
#p.legend.click_policy = 'hide'
#p.legend.label_text_font_size = '5pt'
#show(p)

#Figura 2 Humedad Relativa
#tooltips2 =[('Space','$name'),('Relative Humidity','$y %'),('Date','@x')]
p2 = figure(width=1200,height=306,x_range=p.x_range,x_axis_type='datetime',toolbar_location=None)
p2.yaxis.axis_label = 'Relative Humidity [%]'
#HoverTool.formatters={}
hover2 = HoverTool(
    tooltips=[
        ('Space','$name'),
        #('Relative Humidity','$y %'),
        ('Hour','@datetime{%H:%M}')
    ]
)
#hover1.tooltips = [('Space','$name'),('Temperature','$y °C'),('Date','@$x')]
hover2.formatters = {'@datetime':'datetime'}
p2.add_tools(hover2)
p2.add_tools(crosshair)
for zona in zones:
    if 'AU' in zona:
        p2.line(y='RH_'+zona,x='time',source=source,line_color='blue',name=zona)
    elif 'LAB' in zona:
        p2.line(y='RH_'+zona,x='time',source=source,line_color='red',name=zona)
    else:
        p2.line(y='RH_'+zona,x='time',source=source,line_color='green',name=zona)

models = ["Humphreys & Nicol", "Humphreys", "Griffiths"]

radio_group = RadioGroup(labels=models, active=0,width=5,width_policy='fixed',margin=(0,0,0,-600))
radio_group.js_on_event('button_click', CustomJS(code="""
    console.log('radio_group: active=' + this.origin.active, this.toString())
"""))
# radio_group.labels_text_font_size = '16px'

GHDC = (edificio.Ti_N1AU401[edificio.Ti_N1AU401>edificio.lsc]-edificio.Tn).sum()*(1./6.)
ghdc = Div(width=200,margin=(0,0,0,150),width_policy='fixed')
ghdc.text = '<span style="font-size: 12pt;">Grados hora de disconfort cálido</span><br>' + str(GHDC)


lists = pd.DataFrame()
lists['zones']=zones
lists['ti']=[str(round(edificio['Ti_'+zone].iloc[100],1))+' °C' for zone in zones]
lists['rh']=[str(round(edificio['RH_'+zone].iloc[100],1))+' %' for zone in zones]
lists['aw']=[str(round(edificio['A_totW'+zone].iloc[100]+edificio['A_totV'+zone].iloc[100],1))+'m³' for zone in zones]
lists['x1']=np.ones(22)*0.1
lists['x2']=np.ones(22)*2
lists['x3']=np.ones(22)*3.53
lists['x4']=np.ones(22)*5.1
lists['y']=np.arange(22,0,-1)

titles=pd.DataFrame()
titles['zones']=['Zona']
titles['ti']=['Temperatura al interior']
titles['rh']=['Humedad Relativa']
titles['aw']=['Área de ventanas abierta']
titles['x1']=[0.1]
titles['x2']=[2]
titles['x3']=[3.53]
titles['x4']=[5.1]
titles['y']=[23]

ploting=pd.concat([titles,lists])

listed = ColumnDataSource(ploting)
p3 = figure(width=680,toolbar_location=None,tooltips='',x_range=Range1d(0,6),margin=(-500,0,0,30))

p3.text(x='x2',y='y',text='ti',text_align="center",
        text_font_size="14.5px",
        text_font="arial",
        text_font_style="normal",source=listed)
p3.text(x='x3',y='y',text='rh',text_align="center",
        text_font_size="14.5px",
        text_font="arial",
        text_font_style="normal",source=listed)
p3.text(x='x4',y='y',text='aw',text_align="center",
        text_font_size="14.5px",
        text_font="arial",
        text_font_style="normal",source=listed)
p3.text(x='x1',y='y',text='zones',text_align="left",
        text_font_size="14.5px",
        text_font="arial",
        text_font_style="normal",source=listed)
p3.yaxis.visible = False
p3.xaxis.visible = False
p3.grid.visible = False

comfort_panel=layout([[p,radio_group,ghdc],[p2,p3]])

show(comfort_panel)

In [113]:
lists = pd.DataFrame()
lists['zones']=zones
lists['ti']=[str(round(edificio['Ti_'+zone].iloc[100],1))+' °C' for zone in zones]
lists['rh']=[str(round(edificio['RH_'+zone].iloc[100],1))+' %' for zone in zones]
lists['aw']=[str(round(edificio['A_totW'+zone].iloc[100]+edificio['A_totV'+zone].iloc[100],1))+'m³' for zone in zones]
lists['x1']=np.ones(22)*0.5
lists['x2']=np.ones(22)*1.5
lists['x3']=np.ones(22)*3
lists['x4']=np.ones(22)*4.2
lists['y']=np.arange(22,0,-1)

titles=pd.DataFrame()
titles['zones']=['Zona']
titles['ti']=['Temperatura al interior']
titles['rh']=['Humedad Relativa']
titles['aw']=['Área de ventanas abierta']
titles['x1']=[0.5]
titles['x2']=[1.5]
titles['x3']=[3]
titles['x4']=[4.2]
titles['y']=[23]

ploting=pd.concat([titles,lists])
# ploting.sort_values('y',ascending=False,inplace=True)

In [88]:
ploting

,zones,ti,rh,aw,x1,x2,x3,x4,y
0,Zona,Temperatura al interior,Humedad Relativa,Área de ventanas abierta,0.5,1.5,2.0,2.5,23
0,N1AU401,23.8 °C,23.7 %,31.5m³,0.5,1.5,2.0,2.5,22
1,N1AU402,23.8 °C,23.8 %,30.2m³,0.5,1.5,2.0,2.5,21
2,N1AU403,23.8 °C,23.8 %,33.6m³,0.5,1.5,2.0,2.5,20
3,N1AU404,23.7 °C,23.9 %,27.9m³,0.5,1.5,2.0,2.5,19
4,N2AU101,24.0 °C,24.3 %,26.3m³,0.5,1.5,2.0,2.5,18
5,N2AU102,24.0 °C,24.5 %,26.2m³,0.5,1.5,2.0,2.5,17
6,N2AU103,24.0 °C,24.4 %,31.7m³,0.5,1.5,2.0,2.5,16
7,N2AU201,22.9 °C,26.4 %,28.3m³,0.5,1.5,2.0,2.5,15
8,N2AU202,23.0 °C,26.4 %,28.6m³,0.5,1.5,2.0,2.5,14


In [134]:
lists = pd.DataFrame()
lists['zones']=zones
lists['ti']=[str(round(edificio['Ti_'+zone].iloc[100],1))+' °C' for zone in zones]
lists['rh']=[str(round(edificio['RH_'+zone].iloc[100],1))+' %' for zone in zones]
lists['aw']=[str(round(edificio['A_totW'+zone].iloc[100]+edificio['A_totV'+zone].iloc[100],1))+'m³' for zone in zones]
lists['x1']=np.ones(22)*0.1
lists['x2']=np.ones(22)*2
lists['x3']=np.ones(22)*3.4
lists['x4']=np.ones(22)*5
lists['y']=np.arange(22,0,-1)

titles=pd.DataFrame()
titles['zones']=['Zona']
titles['ti']=['Temperatura al interior']
titles['rh']=['Humedad Relativa']
titles['aw']=['Área de ventanas abierta']
titles['x1']=[0.1]
titles['x2']=[2]
titles['x3']=[3.4]
titles['x4']=[5]
titles['y']=[23]

ploting=pd.concat([titles,lists])

listed = ColumnDataSource(ploting)
p3 = figure(width=900,toolbar_location=None,tooltips='',x_range=Range1d(0,6))

p3.text(x='x2',y='y',text='ti',text_align="center",
        text_font_size="18px",
        text_font="arial",
        text_font_style="normal",source=listed)
p3.text(x='x3',y='y',text='rh',text_align="center",
        text_font_size="18px",
        text_font="arial",
        text_font_style="normal",source=listed)
p3.text(x='x4',y='y',text='aw',text_align="center",
        text_font_size="18px",
        text_font="arial",
        text_font_style="normal",source=listed)
p3.text(x='x1',y='y',text='zones',text_align="left",
        text_font_size="18px",
        text_font="arial",
        text_font_style="normal",source=listed)
p3.yaxis.visible = False
p3.xaxis.visible = False
p3.grid.visible = False

show(p3)